In [5]:
# import external packages
import numpy as np
import pandas as pd
import numba
from numba import vectorize
import glob # for file search
import copy
import os # operating system stuff
import re # regex
import fastparquet # fast read/write for large data structures
import sklearn.preprocessing as pre # for data normalisation
from sklearn.metrics import pairwise_distances

import geopandas as gpd
import rasterio as rio
import rasterio.mask
from rasterio.plot import plotting_extent
from shapely.geometry import Polygon
from shapely.geometry.point import Point
import pyproj
from pyproj import CRS
from inpoly import inpoly2 # for fast inpolygon checks
import utm

import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
from matplotlib import cm as mpl_cm
from matplotlib import colors as mcolors 

from mpl_toolkits.axes_grid1 import make_axes_locatable # for colorbar scaling
from mpl_toolkits.axes_grid1 import ImageGrid
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import FormatStrFormatter

import seaborn as sns
from matplotlib import rc_file_defaults
rc_file_defaults()
# sns.set(style=None, color_codes=True)

from shapely.geometry import Polygon
from shapely.geometry.point import Point
import datetime

import configparser

from cmcrameri import cm # for scientific colourmaps

###########################
# import main local package
import SPOTSAR_main as sm


In [6]:
################ Define user INPUTS #######################
######## please edit the values of this block only ########
###########################################################

# define hillshade file
HS_FILE = './test_data/DEM/TDX_Merapi_WGS84_HS.tif'

# define lon and lat files
LON_FILE = './test_data/CSK_dsc/geo2/20200910.lon'
LAT_FILE = './test_data/CSK_dsc/geo2/20200910.lat'

# define parameter text file
PARAM_FILE = './test_data/CSK_dsc/params.txt'

# define map region of interest
lon_lims = [110.425, 110.45]
lat_lims = [-7.555, -7.535]

# define colour range {min max} (min = -max)
vmax = 3 # range of colourscale in meters

# define file names for data, lon and lat
DIRECTORY_PATH = "./test_data/CSK_dsc/DISP_txt2/"
# define path to ccp and ccs files
DIRECTORY_PATH_CCS = "./test_data/CSK_dsc/CCS2/"

# Set the regular expression pattern to match the file names
PATTERN1 = r"^c20200927_c20201113_disp_[0-9]+_[0-9]+\.txt$"
PATTERN2 = r"^c20200926_c20201113_disp_[0-9]+_[0-9]+\.txt$"
PATTERN3 = r"^c20200927_c20210812_disp_[0-9]+_[0-9]+\.txt$"
PATTERN4 = r"^c20210217_c20210218_disp_[0-9]+_[0-9]+\.txt$"

# Set the regular expression pattern to match the ccs file names
PATTERN_CCS1 = r"^c20200927_c20201113_ccs_[0-9]+_[0-9]+$"
PATTERN_CCS2 = r"^c20200926_c20201113_ccs_[0-9]+_[0-9]+$"
PATTERN_CCS3 = r"^c20200927_c20210812_ccs_[0-9]+_[0-9]+$"
PATTERN_CCS4 = r"^c20210217_c20210218_ccs_[0-9]+_[0-9]+$"


# open hillshade file and re-order offset and CCS files

# open hill shade file with rasterio
DEM_HS = rio.open(HS_FILE)
SHADING = DEM_HS.read(1,masked=True) # rasterio bands are indexed from 1

# extract DEM extent
DEM_EXTENT=[DEM_HS.bounds.left,DEM_HS.bounds.right,DEM_HS.bounds.bottom,DEM_HS.bounds.top]

# read parameters from text file
config = configparser.ConfigParser()
config.read(PARAM_FILE)
WIDTH = int(config.get('params', 'width'))
LINES = int(config.get('params', 'lines'))
WIDTH_CCS = int(config.get('params', 'width_ccs'))
LINES_CCS = int(config.get('params', 'lines_ccs'))
R_START = int(config.get('params', 'r_start'))
A_START = int(config.get('params', 'a_start'))
R_STEP = int(config.get('params', 'r_step'))
A_STEP = int(config.get('params', 'a_step'))
HEADING = float(config.get('params', 'heading'))
MEAN_INC = float(config.get('params', 'mean_inc'))

In [7]:

# reorder file using Post_processing.reorder_files
matching_files1 = sm.Post_processing.reorder_files(DIRECTORY_PATH,PATTERN1,0)
matching_files_ccs1 = sm.Post_processing.reorder_files(DIRECTORY_PATH_CCS,PATTERN_CCS1,0)
matching_files2 = sm.Post_processing.reorder_files(DIRECTORY_PATH,PATTERN2,0)
matching_files_ccs2 = sm.Post_processing.reorder_files(DIRECTORY_PATH_CCS,PATTERN_CCS2,0)
matching_files3 = sm.Post_processing.reorder_files(DIRECTORY_PATH,PATTERN3,0)
matching_files_ccs3 = sm.Post_processing.reorder_files(DIRECTORY_PATH_CCS,PATTERN_CCS3,0)
matching_files4 = sm.Post_processing.reorder_files(DIRECTORY_PATH,PATTERN4,0)
matching_files_ccs4 = sm.Post_processing.reorder_files(DIRECTORY_PATH_CCS,PATTERN_CCS4,0)
matching_files5 = sm.Post_processing.reorder_files(DIRECTORY_PATH+'stor/',PATTERN4,0)
matching_files_ccs5 = sm.Post_processing.reorder_files(DIRECTORY_PATH_CCS+'stor/',PATTERN_CCS4,0)


# test if file ordering has worked
print(matching_files1)
print(matching_files_ccs1)
print(matching_files2)
print(matching_files_ccs2)
print(matching_files3)
print(matching_files_ccs3)
print(matching_files4)
print(matching_files_ccs4)
print(matching_files5)
print(matching_files_ccs5)

['c20200927_c20201113_disp_58_28.txt', 'c20200927_c20201113_disp_140_68.txt', 'c20200927_c20201113_disp_224_108.txt', 'c20200927_c20201113_disp_306_148.txt', 'c20200927_c20201113_disp_388_188.txt']
['c20200927_c20201113_ccs_58_28', 'c20200927_c20201113_ccs_140_68', 'c20200927_c20201113_ccs_224_108', 'c20200927_c20201113_ccs_306_148', 'c20200927_c20201113_ccs_388_188']
['c20200926_c20201113_disp_58_28.txt', 'c20200926_c20201113_disp_140_68.txt', 'c20200926_c20201113_disp_224_108.txt', 'c20200926_c20201113_disp_306_148.txt', 'c20200926_c20201113_disp_388_188.txt']
['c20200926_c20201113_ccs_58_28', 'c20200926_c20201113_ccs_140_68', 'c20200926_c20201113_ccs_224_108', 'c20200926_c20201113_ccs_306_148', 'c20200926_c20201113_ccs_388_188']
['c20200927_c20210812_disp_58_28.txt', 'c20200927_c20210812_disp_140_68.txt', 'c20200927_c20210812_disp_224_108.txt', 'c20200927_c20210812_disp_306_148.txt', 'c20200927_c20210812_disp_388_188.txt']
['c20200927_c20210812_ccs_58_28', 'c20200927_c20210812_ccs_1

In [8]:

# load data from files into class multi-kernel
example_pairs = []
i = 0
for (matching_files,matching_files_ccs) in zip([matching_files1,matching_files2,matching_files3,matching_files4,matching_files5],[matching_files_ccs1,matching_files_ccs2,matching_files_ccs3,matching_files_ccs4,matching_files_ccs5]):
    if i==4:
        datastack = sm.Post_processing.MultiKernel(DIRECTORY_PATH+'stor/',
                                                matching_files,
                                                DIRECTORY_PATH_CCS+'stor/',
                                                matching_files_ccs,
                                                LAT_FILE,
                                                LON_FILE,
                                                HEADING,
                                                MEAN_INC,
                                                LINES_CCS,
                                                WIDTH_CCS)
    else:
        datastack = sm.Post_processing.MultiKernel(DIRECTORY_PATH,
                                                matching_files,
                                                DIRECTORY_PATH_CCS,
                                                matching_files_ccs,
                                                LAT_FILE,
                                                LON_FILE,
                                                HEADING,
                                                MEAN_INC,
                                                LINES_CCS,
                                                WIDTH_CCS)

    # We need to assign some data not stored in the disp.txt files.
    datastack.get_params_from_file_name()
    datastack.get_latlon_from_file(WIDTH)
    datastack.add_lat_lon_to_data(R_START,A_START)
    datastack.crop_stack_ccs(R_STEP,A_STEP)
    # the object datastack now has several attributes associated with the whole dataset (e.g., date1, date2, heading)
    # Next we add all the offset data (disp.txt) to the stack
    stacked_data = datastack.assign_data_to_stack(R_STEP,A_STEP)
    # The attribute 'Stack' we find a list of single-kernel objects which contain the actual offset data, ccp and ccs data and the coordinates.

    # add stack to list
    example_pairs.append(datastack)
    i +=1



In [9]:
# %matplotlib osx
# from scipy.ndimage import generic_filter
# from skimage.morphology import disk
# from skimage import filters
# from scipy.spatial import cKDTree

# filt_size = 3
# filt_radius = 3
# footprint = disk(radius=filt_radius)
# pre_R_non_veg_list = []
# pre_R_veg_list = []
# pre_A_non_veg_list = []
# pre_A_veg_list = []

# for obj in example_pairs[3].Stack:
#     R_off = obj.R_off
#     A_off = obj.A_off

#     print('starting filters')

#     R_off_std = generic_filter(R_off, np.nanstd, footprint=footprint)
#     A_off_std = generic_filter(A_off, np.nanstd, footprint=footprint)

#     print('filters done')


#     DEM_lon_lat = np.column_stack((DEM_lons.flatten(),DEM_lats.flatten()))
#     NDVI_lon_lat = np.column_stack((NDVI_lons.flatten(),NDVI_lats.flatten()))
#     data_lon_lat = np.column_stack((obj.Lon_off.flatten(),obj.Lat_off.flatten()))

#     print('build trees')
#     kdtree_dem = cKDTree(DEM_lon_lat)
#     kdtree_ndvi = cKDTree(NDVI_lon_lat)
#     print('done building trees')

#     data_dem = np.empty(np.shape(R_off.flatten()))
#     data_slope = np.empty(np.shape(R_off.flatten()))
#     data_ndvi = np.empty(np.shape(R_off.flatten()))
#     idx_list = []
#     for i, point in enumerate(data_lon_lat):
#         if not all(np.isnan(point)):
#             print(i)
#             distance,index_DEM = kdtree_dem.query(point)
#             distance,index_NDVI = kdtree_ndvi.query(point)
#             idx_list.append([i,index_DEM,index_NDVI])

#     idx_list = np.asarray(idx_list)
#     data_dem[idx_list[:,0]] = DEM_heights.flatten()[idx_list[:,1]]
#     data_slope[idx_list[:,0]] = DEM_slope.flatten()[idx_list[:,1]]
#     data_ndvi[idx_list[:,0]] = NDVI.flatten()[idx_list[:,2]]
#     data_ndvi[data_ndvi<-1] = np.nan

#     R_sel_non_veg = R_off_std.flatten()[np.argwhere(np.abs(data_ndvi) <= 0.1)]
#     A_sel_non_veg = A_off_std.flatten()[np.argwhere(np.abs(data_ndvi) <= 0.1)]
#     R_sel_veg = R_off_std.flatten()[np.argwhere(data_ndvi >= 0.5)]
#     A_sel_veg = A_off_std.flatten()[np.argwhere(data_ndvi >= 0.5)]

#     R_data_non_veg = R_sel_non_veg.flatten()
#     R_data_veg = R_sel_veg.flatten()
#     R_data_non_veg[R_data_non_veg <= 0] = np.nan
#     R_data_veg[R_data_veg <= 0] = np.nan
#     A_data_non_veg = A_sel_non_veg.flatten()
#     A_data_veg = A_sel_veg.flatten()
#     A_data_non_veg[A_data_non_veg <= 0] = np.nan
#     A_data_veg[A_data_veg <= 0] = np.nan

#     pre_R_non_veg_list.append(R_data_non_veg)
#     pre_R_veg_list.append(R_data_veg)
#     pre_A_non_veg_list.append(A_data_non_veg)
#     pre_A_veg_list.append(A_data_veg)


In [10]:
# load med filt and MKA results from h5 files

import h5py 

# file1 contains window sizes 58, 140, 224, 306, 388, MKA 1-5, MKA 1-3
file1 = './test_data/CSK_dsc/filtered_MKA_offsets/c20200927_c20201113_v3.h5'


# provide attribute names
attr_names1 = ['win_1_Lon', 'win_1_lat', 'win_1_R_off', 'win_1_A_off',
               'win_2_Lon', 'win_2_lat', 'win_2_R_off', 'win_2_A_off',
               'win_3_Lon', 'win_3_lat', 'win_3_R_off', 'win_3_A_off',
               'win_4_Lon', 'win_4_lat', 'win_4_R_off', 'win_4_A_off',
               'win_5_Lon', 'win_5_lat', 'win_5_R_off', 'win_5_A_off',
               'win_13_Lon', 'win_13_lat', 'win_13_R_off', 'win_13_A_off',
               'win_15_Lon', 'win_15_lat', 'win_15_R_off', 'win_15_A_off',]



obj1 = example_pairs[0]

for i, (file, obj,attr_names) in enumerate(zip([file1],[obj1],[attr_names1])):
    f = h5py.File(file,'r')
    for qkey in attr_names:
        attr = np.array(f[f'{qkey}'][:])
        # attr = f.get(f'{qkey}')
        setattr(obj,f'{qkey}',attr)
    f.close() 

In [11]:
print(dir(obj1))

['A_win', 'Ccs_maps', 'Data', 'Data_ccs', 'Date1', 'Date2', 'Filenames', 'Filenames_ccs', 'Heading', 'Lat', 'Lat_file', 'Lat_vec', 'Limits', 'Lon', 'Lon_file', 'Lon_vec', 'Mask_data', 'Mask_data_ccs', 'Mean_inc', 'R_win', 'Run_MKA', 'Run_RSS', 'Stack', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'add_lat_lon_to_data', 'assign_data_to_stack', 'crop_stack_ccs', 'get_latlon_from_file', 'get_params_from_file_name', 'outlier_detection_HDBSCAN_stack', 'outlier_detection_LOF_stack', 'outlier_detection_median_stack', 'query_point_MKA', 'query_point_stack', 'win_13_A_off', 'win_13_Lon', 'win_13_R_off', 'win_13_lat', 'win_15_A_off', 'win_15_Lon', 'win_15_R_off', 'win_15_lat', 'win_1_A_off', 'win_1_Lon

In [12]:
# load external data

import fiona
from matplotlib.colors import ListedColormap
from matplotlib import cm as mpl_cm
## find nearest elevation, slope, and NDVI values
DEM_FILE = './test_data/DEM/TDX_Merapi_WGS84_5m.tif'
SLOPE_FILE = './test_data/DEM/TDX_Merapi_WGS84_5m_slope.tif'
NDVI_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/sentinel2/Sentinel_NDVI_WGS84_v2.tif'

with rio.open(DEM_FILE) as src:
    DEM_heights = src.read(1)
    print('Band1 has shape', DEM_heights.shape)
    height = DEM_heights.shape[0]
    width = DEM_heights.shape[1]
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rio.transform.xy(src.transform, rows, cols)
    DEM_lons = np.array(xs)
    DEM_lats = np.array(ys)
    DEM_extent = [src.bounds.left,src.bounds.right,src.bounds.bottom,src.bounds.top]

with rio.open(SLOPE_FILE) as src:
    DEM_slope = src.read(1)


# define map region of interest
lon_lims = [np.nanmin(DEM_lons), np.nanmax(DEM_lons)]
lat_lims = [np.nanmin(DEM_lats), np.nanmax(DEM_lats)]
crop_flag=1

# create cropping polygon from ROI
if crop_flag:
    coords = ((lon_lims[0], lat_lims[0]), (lon_lims[0], lat_lims[1]), (lon_lims[1], lat_lims[1]), (lon_lims[1], lat_lims[0]), (lon_lims[0], lat_lims[0]))
    crop_poly = Polygon(coords)
    crop_poly_geojson = gpd.GeoSeries([crop_poly])
    crop_poly_geojson.to_file('./test_data/crop_ndvi_poly_v2.shp',crs="EPSG:4326")


with fiona.open('./test_data/crop_ndvi_poly_v2.shp', "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]

# get data and read coords from first file
with rio.open(NDVI_FILE) as src:
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
with rasterio.open("./test_Data/ndvi_cropped.tif", "w", **out_meta) as dest:
    dest.write(out_image)

with rio.open("./test_Data/ndvi_cropped.tif") as src:
    NDVI = src.read(1)
    print('Band1 has shape', NDVI.shape)
    height = NDVI.shape[0]
    width = NDVI.shape[1]
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rio.transform.xy(src.transform, rows, cols)
    NDVI_lons = np.array(xs)
    NDVI_lats = np.array(ys)
    NDVI_extent = [src.bounds.left,src.bounds.right,src.bounds.bottom,src.bounds.top]

# define colormaps
x= mpl_cm.get_cmap('Blues_r', 135)
y= mpl_cm.get_cmap('YlGn', 135)
z = np.vstack((x(range(135)),
                       y(range(135))))
ndvi_cmap = ListedColormap(z, name='BlYlGn')

Band1 has shape (2028, 1672)


Band1 has shape (171, 141)


/var/folders/m9/32v45xkn4zx6js6htb1prb2r0000gn/T/ipykernel_79289/3334344435.py:65: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  x= mpl_cm.get_cmap('Blues_r', 135)
/var/folders/m9/32v45xkn4zx6js6htb1prb2r0000gn/T/ipykernel_79289/3334344435.py:66: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  y= mpl_cm.get_cmap('YlGn', 135)


In [14]:

# lava flow files:
L1888_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/merapi_maps/L1888_v2.shp'
L1948_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/merapi_maps/L1948.shp'
L1956_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/merapi_maps/L1956.shp'
L1992_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/merapi_maps/L1992.shp'
L1997_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/merapi_maps/L1997.shp'
L1998_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/merapi_maps/L1998.shp'
CRATER_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/merapi_maps/Merapi_crater.shp'

# Read the shapefile
L1888 = gpd.read_file(L1888_FILE)
L1948 = gpd.read_file(L1948_FILE)
L1956 = gpd.read_file(L1956_FILE)
L1992 = gpd.read_file(L1992_FILE)
L1997 = gpd.read_file(L1997_FILE)
L1998 = gpd.read_file(L1998_FILE)
CRATER = gpd.read_file(CRATER_FILE)

# Extract latitude and longitude into separate columns
coords_L1888 = np.array(list(L1888["geometry"][0].coords))
coords_L1956_1 = np.array(list(L1956["geometry"][0].coords))
coords_L1956_2 = np.array(list(L1956["geometry"][1].coords))
coords_L1948 = np.array(list(L1948["geometry"][0].coords))
coords_L1992 = np.array(list(L1992["geometry"][0].coords))
coords_L1997 = np.array(list(L1997["geometry"][0].coords))
coords_L1998 = np.array(list(L1998["geometry"][0].coords))
coords_CRATER = np.array(list(CRATER["geometry"][0].coords))

In [15]:
%matplotlib osx

import string
## plot results before and after outlier removal

def plot_offset(ax, Lon_off,Lat_off, arr, grid_size,cmap, clims,lon_lims,lat_lims):
    plot_data = ax.hexbin(Lon_off.flatten(),Lat_off.flatten(),C=arr.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
    ax.set_xlim(lon_lims)
    ax.set_ylim(lat_lims)
    ax.set_aspect('equal', 'box')
    ax.add_artist(ScaleBar(sm.plot.get_1deg_dist(),location='lower left'))
    return plot_data, ax



fig = plt.figure(figsize=(10,8))
gs = GridSpec(5, 4, figure=fig)
ax00 = fig.add_subplot(gs[0, 0]) # win 1 R off pre
ax10 = fig.add_subplot(gs[1, 0]) # win 2 R off pre
ax20 = fig.add_subplot(gs[2, 0]) # win 3 R off pre
ax30 = fig.add_subplot(gs[3, 0]) # win 4 R off pre
ax40 = fig.add_subplot(gs[4, 0]) # win 5 R off pre

ax01 = fig.add_subplot(gs[0, 1]) # win 1 A off pre
ax11 = fig.add_subplot(gs[1, 1]) # win 2 A off pre
ax21 = fig.add_subplot(gs[2, 1]) # win 3 A off pre
ax31 = fig.add_subplot(gs[3, 1]) # win 4 A off pre
ax41 = fig.add_subplot(gs[4, 1]) # win 5 A off pre


ax02 = fig.add_subplot(gs[0, 2]) # win 1 R off post
ax12 = fig.add_subplot(gs[1, 2]) # win 2 R off post
ax22 = fig.add_subplot(gs[2, 2]) # win 3 R off post
ax32 = fig.add_subplot(gs[3, 2]) # win 4 R off post
ax42 = fig.add_subplot(gs[4, 2]) # win 5 R off post

ax03 = fig.add_subplot(gs[0, 3]) # win 1 A off post
ax13 = fig.add_subplot(gs[1, 3]) # win 2 A off post
ax23 = fig.add_subplot(gs[2, 3]) # win 3 A off post
ax33 = fig.add_subplot(gs[3, 3]) # win 4 A off post
ax43 = fig.add_subplot(gs[4, 3]) # win 5 A off post

grid_size = 1000
clims = [-3,3]
cmap = cm.vik
lon_lims = [110.4313, 110.4496]
lat_lims = [-7.5463, -7.5350]

plot_offset(ax00, obj1.Stack[0].Lon_off,obj1.Stack[0].Lat_off,obj1.Stack[0].R_off, grid_size,cmap, clims,lon_lims,lat_lims)
plot_offset(ax10, obj1.Stack[1].Lon_off,obj1.Stack[1].Lat_off,obj1.Stack[1].R_off, grid_size,cmap, clims,lon_lims,lat_lims)
plot_offset(ax20, obj1.Stack[2].Lon_off,obj1.Stack[2].Lat_off,obj1.Stack[2].R_off, grid_size,cmap, clims,lon_lims,lat_lims)
plot_offset(ax30, obj1.Stack[3].Lon_off,obj1.Stack[3].Lat_off,obj1.Stack[3].R_off, grid_size,cmap, clims,lon_lims,lat_lims)
plot_offset(ax40, obj1.Stack[4].Lon_off,obj1.Stack[4].Lat_off,obj1.Stack[4].R_off, grid_size,cmap, clims,lon_lims,lat_lims)

plot_offset(ax01, obj1.win_1_Lon,obj1.win_1_lat,obj1.win_1_R_off, grid_size,cmap, clims,lon_lims,lat_lims)
plot_offset(ax11, obj1.win_2_Lon,obj1.win_2_lat,obj1.win_2_R_off, grid_size,cmap, clims,lon_lims,lat_lims)
plot_offset(ax21, obj1.win_3_Lon,obj1.win_3_lat,obj1.win_3_R_off, grid_size,cmap, clims,lon_lims,lat_lims)
plot_offset(ax31, obj1.win_4_Lon,obj1.win_4_lat,obj1.win_4_R_off, grid_size,cmap, clims,lon_lims,lat_lims)
plot_offset(ax41, obj1.win_5_Lon,obj1.win_5_lat,obj1.win_5_R_off, grid_size,cmap, clims,lon_lims,lat_lims)


plot_offset(ax02, obj1.Stack[0].Lon_off,obj1.Stack[0].Lat_off,obj1.Stack[0].A_off, grid_size,cmap, clims,lon_lims,lat_lims)
plot_offset(ax12, obj1.Stack[1].Lon_off,obj1.Stack[1].Lat_off,obj1.Stack[1].A_off, grid_size,cmap, clims,lon_lims,lat_lims)
plot_offset(ax22, obj1.Stack[2].Lon_off,obj1.Stack[2].Lat_off,obj1.Stack[2].A_off, grid_size,cmap, clims,lon_lims,lat_lims)
plot_offset(ax32, obj1.Stack[3].Lon_off,obj1.Stack[3].Lat_off,obj1.Stack[3].A_off, grid_size,cmap, clims,lon_lims,lat_lims)
plot_offset(ax42, obj1.Stack[4].Lon_off,obj1.Stack[4].Lat_off,obj1.Stack[4].A_off, grid_size,cmap, clims,lon_lims,lat_lims)


plot_offset(ax03, obj1.win_1_Lon,obj1.win_1_lat,obj1.win_1_A_off, grid_size,cmap, clims,lon_lims,lat_lims)
plot_offset(ax13, obj1.win_2_Lon,obj1.win_2_lat,obj1.win_2_A_off, grid_size,cmap, clims,lon_lims,lat_lims)
data_for_colourbar = plot_offset(ax23, obj1.win_3_Lon,obj1.win_3_lat,obj1.win_3_A_off, grid_size,cmap, clims,lon_lims,lat_lims)
plot_offset(ax33, obj1.win_4_Lon,obj1.win_4_lat,obj1.win_4_A_off, grid_size,cmap, clims,lon_lims,lat_lims)
plot_offset(ax43, obj1.win_5_Lon,obj1.win_5_lat,obj1.win_5_A_off, grid_size,cmap, clims,lon_lims,lat_lims)


bbox = dict(fc="1.0")
for (a, t) in zip(fig.axes,list(string.ascii_uppercase)):
    a.get_xaxis().set_visible(False)
    # a.get_yaxis().set_visible(False)
    a.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    left=False,
    right=False,
    labelleft=False,
    labelbottom=False) # labels along the bottom edge are off
    a.annotate(t,xy=(0.03,0.83),xycoords='axes fraction',fontsize=12, bbox=bbox)
    a.plot(coords_CRATER[:,0],coords_CRATER[:,1],linewidth=2,color='black',linestyle='--',label='Crater rim')


# fig.tight_layout()

ax00.set_ylabel('Window size:\n20 m',fontsize=12)
ax10.set_ylabel('Window size:\n48 m',fontsize=12)
ax20.set_ylabel('Window size:\n75 m',fontsize=12)
ax30.set_ylabel('Window size:\n103 m',fontsize=12)
ax40.set_ylabel('Window size:\n130 m',fontsize=12)

ax00.set_title('Original range offset')
ax01.set_title('Filtered range offset')
ax02.set_title('Original azimuth offset')
ax03.set_title('Filtered azimuth offset')

# add colourbar axis
import matplotlib.patches as mpatches
import matplotlib.transforms as mtransforms
def add_bottom_cax(ax, pad, width):
    axpos = ax.get_position()
    caxpos = mtransforms.Bbox.from_extents(
        axpos.x0,
        axpos.y1-pad-width,
        axpos.x1,
        axpos.y1-pad
    )
    cax = ax.figure.add_axes(caxpos)

    return cax
pad = 0.02
width = 0.02
ax40_pos = ax40.get_position()
ax43_pos = ax43.get_position()
caxpos = mtransforms.Bbox.from_extents(
    ax40_pos.x0,
    ax40_pos.y0-pad-width,
    ax43_pos.x1,
    ax43_pos.y0-pad
)
cax = fig.add_axes(caxpos)

# cax = add_bottom_cax(ax23, pad=0.02, width=0.02)

cbar = fig.colorbar(data_for_colourbar[0], cax=cax,label='Range or azimuth offset [m]', orientation = 'horizontal')
cbar.set_label(label='Range or azimuth offset [m]', size=12)
fig.subplots_adjust(wspace=0.05, hspace=0.05)
fig.savefig('/Users/markbemelmans/Documents/PhD/projects/Merapi2021/figures/outlier_removal_before_after.png',dpi=300)
# fig.savefig('/Users/markbemelmans/Documents/PhD/projects/Merapi2021/figures/outlier_removal_before_after.pdf')

: 

In [ ]:
plt.close('all')

: 